In [ ]:
from TranscriptionDataset.ArrangementDataset import ArrangementDataModule
from TranscriptionModel.ArrangementModel import ArrangementModel
from pytorch_lightning import Trainer,seed_everything
import torch
import matplotlib.pyplot as plt
from pytorch_lightning.loggers import WandbLogger
import os

plt.rcParams['figure.figsize'] = [12, 8]
print(f"Cuda : {torch.cuda.is_available()}")
torch.set_float32_matmul_precision('medium')

seed_everything(42, workers=True)
# os.environ['WANDB_MODE'] = 'offline'

In [ ]:
SAMPLE_RATE = 16000
dataset = "Trainsets/massive.hdf5"

In [ ]:
data_module = ArrangementDataModule(
    location=dataset,
    sample_rate=SAMPLE_RATE,
    val_size=0.1,
    num_workers=4,
    batch_size=16,
)

In [ ]:
model = ArrangementModel(
    tuning_size=data_module.transform.tuning_output_size,
    capo_size=data_module.transform.capo_output_size,
    arrangement_size=data_module.transform.arrangement_output_size
)

In [ ]:
# train
wandb_logger = WandbLogger(
    project="ArrangementModel"
)
trainer = Trainer(
    default_root_dir="model_weights/",
    accelerator="gpu",
    devices=-1,
    max_epochs=5,
    logger=wandb_logger
)
# trainer = Trainer()
trainer.fit(model, data_module)
# wandb_logger.finish()